In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import keras.backend as K
from keras.layers import Input, Layer
from keras.models import Model
from keras.models import load_model
from tensorflow.keras.optimizers import Adam

import operator
from typing import Union, Tuple, List, Any, Optional

2021-12-20 10:54:15.529779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-20 10:54:15.529866: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
d = {'a': 1, 'b': 2, 'c': 3}
ser = pd.Series(data=d, index=['a', 'b', 'c'])
list(ser.index)

['a', 'b', 'c']

In [2]:
from src.counterfactuals.base import CounterfactualMethod

#from src.counterfactuals.constraints import Freeze, OneHot

In [103]:
class Cadex(CounterfactualMethod):
    '''
    Creates a counterfactual explanation based on a pre-trained model using CADEX method
    The model has to be a Keras classifier model, where in the final classification layer, each class label must
    have a separate unit.
    '''
    def __init__(self, pretrained_model, constraints: Optional[List[Any]] = None) -> None:
        self.model = pretrained_model 
        self._constraints = constraints if constraints is not None else [] 
        
        self.x = None
        self.y_expected = None
        self.y_expected_class = None
        self.mask = None
        self.C = None
        
        
    def generate(self, x: Union[pd.Series, np.ndarray], max_epoch=1000, threshold=0.5) -> Union[pd.DataFrame, np.ndarray]:
        self.x = tf.Variable(x, dtype=tf.float32)
        y_original = self._get_predicted_class(self.x)
        self.y_expected_class = abs(y_original - 1)
        if y_original == 0:
            self.y_expected = tf.constant([[0, 1]], dtype=tf.float32)
        else:
            self.y_expected = tf.constant([[1, 0]], dtype=tf.float32)

        opt = Adam()
        
        input_shape = self.x.shape[1:]
        self._initialize_mask(input_shape)
        self._initialize_C(input_shape)
        
        for _ in range(max_epoch):
            gradients = self._get_gradient()
            opt.apply_gradients(zip([gradients], [self.x]))
            x_corrected = self._correct_categoricals(threshold)
            if self._get_predicted_class(x_corrected) == self.y_expected_class:
                return x_corrected
            
            
    def _get_predicted_class(self, x: tf.Variable):
        return self.model(x).numpy().argmax()
    
    def _correct_categoricals(self, threshold):
        corrected_x = self.x.numpy()[0]
        for constraint in self._constraints:
            if isinstance(constraint, OneHot):
                # if second best is bigger than threshold than flip
                feature = corrected_x[constraint.start_column, constraint.end_column]
                sorted_features = sorted(zip(enumerate(feature)), key=lambda feat: feat[1], reverse=True)
                if sorted_features[1][1] > threshold:
                    corrected_x[constraint.start_column, constraint.end_column] = 0
                    corrected_x[constraint.start_column + sorted_features[1][0]] = 1
                    
                else:
                    corrected_x[constraint.start_column, constraint.end_column] = 0
                    corrected_x[constraint.start_column + sorted_features[0][0]] = 1
                    
        return tf.convert_to_tensor([corrected_x])       
    
    def _update_mask(self, gradient):
        new_mask = self.mask.copy()
        for i in range(len(gradient)):
            if not((self.C[i] > 0 and gradient[i] < 0) or (self.C[i] < 0 and gradient[i] > 0) or self.C[i] == 0):
                new_mask[i] = 0
        return new_mask
            
    def _get_gradient(self):           
        with tf.GradientTape() as t:
            t.watch(self.x)
            y_pred = model(self.x)
            loss = tf.keras.losses.categorical_crossentropy(self.y_expected, y_pred)

        gradients = t.gradient(loss, self.x).numpy().flatten()
        updated_mask = self._update_mask(gradients)
        return tf.convert_to_tensor([gradients * updated_mask])

    def _initialize_mask(self, shape, dtype="float32") -> np.ndarray:
        self.mask = np.ones(shape, dtype=dtype)
        for constraint in self._constraints:
            if isinstance(constraint, Freeze):
                for column in constraint.columns:
                    self.mask[column] = 0
        
            
    def _initialize_C(self, shape, dtype="float32") -> np.ndarray:
        self.C = np.zeros(shape, dtype=dtype)
        for constraint in self._constraints:
            if isinstance(constraint, ValueChangeDirection):
                val = 1 if constraint.direction == "+" else -1
                for column in constraint.columns:
                    self.C[column] = val
    

In [98]:
from data.LoanData import LoanData

data = LoanData(input_file='data/input_german.csv', labels_file='data/labels_german.csv')
model = load_model('data/model_german.h5')

In [106]:
input_data = data.valid_input.iloc[[9], :]

cadex = Cadex(model)
cf = cadex.generate(input_data)

In [107]:
cf

<tf.Tensor: shape=(1, 61), dtype=float32, numpy=
array([[ 0.7022602 ,  0.6829853 ,  0.42078522, -0.31666508, -0.12075999,
        -1.1929638 , -0.9194117 , -0.98668694,  1.1124228 ,  0.15679565,
        -0.21944797, -0.6566915 , -0.01448072,  3.6497478 , -0.6263946 ,
        -0.62785643, -0.8104569 , -0.9861811 ,  0.16849802,  0.26206273,
         3.7297518 ,  0.03780277, -0.7204713 , -0.14534009,  0.36970177,
         0.36677936, -0.7768779 , -0.7850558 ,  0.32007104,  0.17918716,
        -0.00805103, -1.0595104 , -0.64736927,  0.16347276,  1.3903598 ,
        -0.66945934,  0.2855454 , -0.17190108,  1.4184916 , -0.24126053,
        -0.16052814, -0.07783373, -0.8193088 , -1.6210183 ,  4.12619   ,
         0.06565855,  1.1849915 , -0.8532869 ,  0.0794995 ,  0.27006158,
        -0.17303558, -0.84807295, -1.6268035 ,  1.7141602 ,  0.2732813 ,
        -0.19223668,  0.92396384,  0.05571455, -0.84632087, -0.09522267,
        -0.19433047]], dtype=float32)>

In [102]:
model(cf)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.49955207, 0.5004479 ]], dtype=float32)>

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 15)                930       
                                                                 
 dense_1 (Dense)             (None, 2)                 32        
                                                                 
Total params: 962
Trainable params: 962
Non-trainable params: 0
_________________________________________________________________


In [72]:
model = load_model('data/model_german.h5')
# for layer in model.layers[1:]:
#     layer.trainable = False

input_data = data.valid_input.iloc[[9], :]
y = 0

x_tensor = tf.Variable(input_data, dtype=tf.float32)
target = tf.constant([[1, 0]], dtype=tf.float32)
with tf.GradientTape() as t:
    t.watch(x_tensor)
    pred = model(x_tensor, training=False)
    loss = tf.keras.losses.categorical_crossentropy(target, pred)

gradients = t.gradient(loss, x_tensor)
print(gradients)

tf.Tensor(
[[-2.2264458e-01 -8.7308809e-02 -2.0462963e-01 -6.0249632e-03
   1.2517105e-01 -1.4757029e-02 -5.4372768e-03 -2.0502799e-03
  -9.8711647e-02  2.7196212e-03  7.8916734e-01 -5.9503266e-03
   3.4428129e-01  7.8258254e-03  9.3462528e-04 -1.5739987e-02
  -9.9351825e-03 -2.8663009e-01  1.5845481e-01 -1.3930327e-04
  -1.6606465e-02  4.5048827e-03 -6.7301287e-04  2.9692695e-02
   1.4130946e-03  1.1004622e-03 -8.7195439e-03 -5.6870906e-03
  -1.9781412e-01  5.5457042e-03  3.9115865e-02 -7.3289745e-02
  -5.6496391e-04  1.9869458e-02  9.9704154e-02 -1.6209289e-02
   3.4746444e-03  1.5288955e-03 -5.7453904e-03  2.2431049e-03
   7.1861349e-02  9.8289317e-04 -5.9918296e-03  2.7083582e-01
  -4.9224366e-03  7.2972020e-03  7.0662096e-02 -4.9804062e-02
   9.0465089e-03 -1.9508768e-02 -9.2928894e-05 -3.4295744e-03
  -3.9709095e-02  1.0808980e-02  1.0363033e-02 -1.4564778e-01
  -5.2685407e-03  2.1362123e-01 -5.7223402e-02  3.3521783e-03
  -1.5558908e-04]], shape=(1, 61), dtype=float32)


In [34]:
pred = model(x_tensor)
pred

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.584158  , 0.41584197]], dtype=float32)>

In [25]:
x_tensor

<tf.Variable 'Variable:0' shape=(1, 61) dtype=float32, numpy=
array([[ 1.2939999 ,  2.238576  , -1.7892507 ,  1.031015  , -0.6676145 ,
        -0.75488985, -0.44553548,  1.6173494 , -0.6086458 , -0.26162457,
        -0.7980154 , -0.23686624, -0.6595283 , -0.31268218,  0.97225964,
        -0.20577969, -0.34194803, -0.5436419 ,  2.830916  , -0.10515917,
        -0.21786045, -0.46802524, -0.09728166, -0.61588174, -0.16518874,
        -0.09728166, -0.31268218, -0.26162457,  0.81119114, -0.26162457,
        -0.46802524, -0.66685474, -0.22941573,  3.294215  , -1.1231581 ,
        -0.1973855 , -0.2256172 ,  0.30662206, -0.56291425,  1.4258946 ,
        -0.62554324, -0.41750678, -0.38602147,  0.46305642, -0.22176638,
        -0.32751554, -1.6300745 ,  2.159564  ,  2.5211747 , -1.3534616 ,
        -0.14954671, -0.49266464, -1.2613854 ,  1.2613854 , -0.20987062,
         0.20987062, -0.7337994 , -0.45056355,  2.2194426 , -0.58937967,
        -0.24052285]], dtype=float32)>

In [26]:
opt = Adam()
opt.apply_gradients(zip([gradients], [x_tensor]))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [27]:
x_tensor

<tf.Variable 'Variable:0' shape=(1, 61) dtype=float32, numpy=
array([[ 1.2949998 ,  2.2395759 , -1.7882508 ,  1.0320128 , -0.6686126 ,
        -0.75389075, -0.44454736,  1.616385  , -0.6076458 , -0.26261377,
        -0.7990154 , -0.2358703 , -0.6605273 , -0.31368026,  0.9712868 ,
        -0.20478083, -0.3409492 , -0.542642  ,  2.8299162 , -0.1061469 ,
        -0.21686082, -0.4690224 , -0.09630696, -0.6168814 , -0.16618188,
        -0.09634716, -0.3116846 , -0.2606278 ,  0.8121911 , -0.26261476,
        -0.46902463, -0.66585493, -0.22842167,  3.293216  , -1.1241522 ,
        -0.19638936, -0.22661312,  0.30762202, -0.56193703,  1.4248993 ,
        -0.62654215, -0.41828948, -0.38504106,  0.4620583 , -0.2207763 ,
        -0.32851177, -1.6310711 ,  2.160563  ,  2.5201786 , -1.3524619 ,
        -0.15040581, -0.49365318, -1.2603914 ,  1.2603877 , -0.21086782,
         0.210859  , -0.73280245, -0.45156303,  2.2204347 , -0.59037644,
        -0.23954919]], dtype=float32)>